In [253]:
import sys
import os
import yaml
import json
import math
import time
import statistics as stats
from datetime import datetime
from elasticsearch import Elasticsearch
from ssl import create_default_context
import es_monitor_helpers as helpers
import es_monitor_stat_tables as stat_tables

def getESConnection():
    with open('env_config_rf3prodpoc.yml', 'r') as ymlfile:
        env_config = yaml.load(ymlfile, Loader=yaml.SafeLoader)

    context = create_default_context(cafile=env_config['ssl_ca'])
    es = Elasticsearch(env_config['elk_hosts'],
        http_auth=(env_config['elk_user'], env_config['elk_pass']),
        scheme="https",
        port=env_config['elk_port'],
        ssl_context=context,
        retry_on_timeout = True,
        timeout=360
    )
    
    #print(es.cluster.health())
    #print("INFO\n",es.info())
    
    return es

def formquery(time,entity=''):
    query = '''
{
  "version": true,
  "size": 10000,
  "sort": [
    {
      "LogLineTimeStamp": {
        "order": "desc",
        "unmapped_type": "boolean"
      }
    }
  ],
  "_source": {
    "excludes": []
  },
  "aggs": {
    "2": {
      "date_histogram": {
        "field": "LogLineTimeStamp",
        "fixed_interval": "30s",
        "time_zone": "America/Los_Angeles",
        "min_doc_count": 1
      }
    }
  },
  "docvalue_fields": [
    {
      "field": "@timestamp",
      "format": "date_time"
    },
    {
      "field": "LogLineTimeStamp",
      "format": "date_time"
    }
  ],
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "bool": {
            "filter": [
              {
                "bool": {
                  "should": [
                    {
                      "bool": {
                        "should": [
                          {
                            "match_phrase": {
                              "LogMessage": "Changing state to DELETED;"
                            }
                          }
                        ],
                        "minimum_should_match": 1
                      }
                    },
                    {
                      "bool": {
                        "should": [
                          {
                            "bool": {
                              "should": [
                                {
                                  "match_phrase": {
                                    "LogMessage": "Status Change: Getting information for lot."
                                  }
                                }
                              ],
                              "minimum_should_match": 1
                            }
                          },
                          {
                            "bool": {
                              "should": [
                                {
                                  "match_phrase": {
                                    "LogMessage": "Changing state"
                                  }
                                }
                              ],
                              "minimum_should_match": 1
                            }
                          }
                        ],
                        "minimum_should_match": 1
                      }
                    }
                  ],
                  "minimum_should_match": 1
                }
              },
              {
                "bool": {
                  "should": [
                    {
                      "query_string": {
                        "fields": [
                          "LogSource"
                        ],
                        "query": "IBWFService*"
                      }
                    }
                  ],
                  "minimum_should_match": 1
                }
              }
            ]
          }
        },subent
        {
          "range": {
            "LogLineTimeStamp": {
              "format": "strict_date_optional_time",
              "gte": "now-subtime",
              "lte": "now"
            }
          }
        }
      ]
    }
  }
}
    '''
    subq = query.replace('subtime',time)
    subq1 = subq.replace('subent',entity)
    return subq1

def ent_query(entity):
    query ='''
        {
          "bool": {
            "should": [
              {
                "match_phrase": {
                  "Entity.keyword": "subent"
                }
              }
            ],
            "minimum_should_match": 1
          }
        },
    '''
    return query.replace('subent',entity)

def preprocess(data):
    if len(data) > 1:
        for i in range(0,len(data)-1):
            if data[i] > 0:
                if data[i+1] == 1:
                    data[i+1] = data[i] + 1

    return(data)


def index_doc(cluster, doc, es):
    t = time.gmtime()
    timestamp = datetime(
        t.tm_year, 
        t.tm_mon, 
        t.tm_mday, 
        t.tm_hour, 
        t.tm_min, 
        t.tm_sec
        ).isoformat()
    index = "rf3.amo.rule.perf-{}".format(
        datetime.now().strftime("%Y-%m"))
    
    intervals = doc["intervals"]
    for interval in intervals:
        mu = intervals[interval]["mu"]
        alpha = intervals[interval]["alpha"]
        beta = intervals[interval]["beta"]
        if alpha == 1:
            sigma_2 = 0
        else:
            sigma_2 = beta/(alpha - 1)
        

    body = {
        "timestamp": timestamp,
        "cluster": cluster,
        "alpha": doc["alpha"],
        "beta": doc["beta"],
        "lambda": doc["lambda"],
        "fail_prob": doc["fail_prob"],
        "max_consec": doc["max_consec"],
        "prob": doc["prob"],
        "intervals": intervals
    }

    es.index(index=index, body=body)

#def main():
#    es = getESConnection()
#    pattern = "rf3.winclient.opsview.toolmessages*"
#    query = formquery()
#    response = es.search(index=pattern, body=query)
#    docs = response['hits']['hits']
#
#    tracker = 'tracker_amo_rule_perf_main.json'
#    if os.path.exists(tracker):
#        with open(tracker, 'r') as f:
#            clusters = json.load(f)
#    else: 
#        clusters = {}



if __name__ == "__main__":
    tracker = "tracker_errors.json"
    id = "es_monitor_amo_rule_perf"
    try:
        main()
        helpers.throttle(id, tracker, False)
    except Exception as e:
        message = str(e)
        subject = "{} error".format(id)
        
        if helpers.throttle(id, tracker, True):
            helpers.test_page(subject, message)

In [255]:
import pprint
import re
es = getESConnection()
pattern = "rf3.winclient.opsview.toolmessages*"
query = formquery('10m')
response = es.search(index=pattern, body=query)
docs = response['hits']['hits']

def get_sc_states(docs):
    dic = {}
    for doc in docs:
        lot = ''
        log_time = doc["_source"]["LogLineTimeStamp"]
        entity = doc["_source"]["Entity"]
        log_message = doc["_source"]["LogMessage"]
        if re.match('^Status Change: Getting information for lot',log_message):
            p = re.compile("lot (\w+)")
            result = p.search(log_message)
            lot = result.group(1)
        if re.match('^Changing state to',log_message):
            p = re.compile("to (\w+\*?\*?)")
            result = p.search(log_message)
            batch_state = result.group(1)  
        log = (log_time,lot,log_message,batch_state)
        if entity not in dic.keys():
            dic[entity] = [log]
        else:
            dic.get(entity).append(log)
            
    return dic
    #pprint.pprint(dic)

dic = get_sc_states(docs)
#pprint.pprint(dic)

def get_last_lot(tup_list):
    for row in tup_list:
        lot = row[1]
        if lot:
            break
    return lot
        
dic2 = {}
for entity in dic.keys():
    batch_state = dic[entity][0][3]
    if re.match('\w+\*\*',batch_state):
        lot = get_last_lot(dic[entity])
        dic2[entity] = (batch_state,lot)

#print('Batches needing recovery:')
pprint.pprint(dic2)

{'HTP414': ('RU**', ''),
 'OMB466': ('RU**', 'D945CTN0'),
 'SNU09': ('MN**', 'D941257B'),
 'TGR403': ('RU**', ''),
 'UDC417': ('RU**', '')}


In [256]:
dic3 = {}
dic3 = dic2
for entity in dic2.keys():
    if not dic2[entity][1]:
        query = formquery('3h',ent_query(entity))
        response = es.search(index=pattern, body=query)
        docs = response['hits']['hits']
        dic = get_sc_states(docs)
        lot = get_last_lot(dic[entity])
        batch_state = dic[entity][0][3]
        if batch_state != 'DELETED':
            dic3[entity] = (batch_state,lot)
pprint.pprint(dic3)

{'HTP414': ('RU**', 'W9488460'),
 'OMB466': ('RU**', 'D945CTN0'),
 'SNU09': ('MN**', 'D941257B'),
 'TGR403': ('RU**', 'Z821TBK0'),
 'UDC417': ('RU**', 'D734TSZ0')}


In [233]:
dic

{'SST01': [('2019-12-16T15:52:18.714Z', '', 'Changing state to RU**;', 'RU**'),
  ('2019-12-16T15:52:15.705Z', '', 'Changing state to RU**;', 'RU**'),
  ('2019-12-16T15:49:26.969Z', '', 'Changing state to RUN ;', 'RUN'),
  ('2019-12-16T15:49:20.369Z', '', 'Changing state to EXEC;', 'EXEC'),
  ('2019-12-16T15:47:30.089Z', '', 'Changing state to RUN ;', 'RUN'),
  ('2019-12-16T15:47:23.448Z', '', 'Changing state to EXEC;', 'EXEC'),
  ('2019-12-16T15:46:35.891Z', '', 'Changing state to MNT ;', 'MNT'),
  ('2019-12-16T15:46:34.710Z', '', 'Changing state to SU  ;', 'SU'),
  ('2019-12-16T15:46:34.218Z', '', 'Changing state to INTR;', 'INTR'),
  ('2019-12-16T15:46:21.338Z',
   'H9424910',
   'Status Change: Getting information for lot H9424910;',
   'INTR'),
  ('2019-12-16T15:46:21.293Z',
   'H9424910',
   'Status Change: Getting information for lot H9424910;',
   'INTR'),
  ('2019-12-16T15:46:11.470Z', '', 'Changing state to DELETED;', 'DELETED'),
  ('2019-12-16T15:45:07.310Z', '', 'Changing s